In [ ]:
from IPython.core.display import HTML
with open("style.html", "r") as file:
    css = file.read()
HTML(css)

# Grafische Oberfläche

In diesem Notebook ist das Spielen in der GUI implementiert. Die Hilfsfunktionen sind in dem Notebook nmm-game-utils definiert.

In [ ]:
%run ./nmm-gui-utils.ipynb

## Klasse GameState

Die Klasse `GameState` dient zum Spielen und Verwalten von einem Mühle-Spiel in der GUI.

Der Konstruktur der Klasse hat elf Eingabeparameter, die alle optional sind:

- `state` ist der Startzustand für das Spiel. Standardmäßig wird das Spiel mit `s0` gestartet, welches ein leeres Spielfeld darstellt;
- `player` definiert den Spieler, der den ersten Zug spielt. Standardmäßig wird das Spiel mit dem weißen Spieler (`w`) gestartet;
- `algorithm1` definiert, ob und welcher Algorithmus für den weißen Spieler spielt. Standardmäßig wird der Spieler von einem Menschen gespielt (`None`). Für _α-β-Pruning_ ist eine Instanz der Klasse `AlphaBetaPruning` zu übergeben und für _Minimax_ eine Instanz der Klasse `Minimax`.
- `algorithm2` definiert, ob und welcher Algorithmus für den schwarzen Spieler spielt. Standardmäßig wird der Spieler von einem Menschen gespielt (`None`). Für _α-β-Pruning_ ist eine Instanz der Klasse `AlphaBetaPruning` zu übergeben und für _Minimax_ eine Instanz der Klasse `Minimax`.
- `timeout` definiert einen Timeout in Sekunden, der nach einem Computer-Zug gesetzt wird. Dies dient dazu, die Übersichtlichkeit zu erhöhen, wenn Computer gegen Computer spielt. Standardmäßig gibt es keinen Timeout (`None`);
- `stepwise` ist ein boolischer Wert und gibt an, ob das Spiel bei Computer gegen Computer im Einzelschrittmodus gespielt wird. Das bedeutet, nach jedem Computerzug muss der nächste Computerzug manuell begonnen werden. Standardmäßig ist der Einzelschrittmodus deaktiviert (`False`);
- `limitMovesWithoutMill` ist eine Ganzzahl, die angibt, wie viele Züge ohne geschlagende Mühle gespielt werden können. Ist das Limit überschritten, endet das Spiel in einem Unentschieden. Der Standardwert ist `30`. Um dieses Limit auszuschalten, muss der Wert auf `None` gesetzt werden;
- `limitStatesCounter` ist eine Ganzzahl, die angibt, wie oft ein gleicher Zustand gespielt werden kann. Ist das Limit überschritten, endet das Spiel in einem Unentschieden. Der Standardwert ist `5`. Um dieses Limit auszuschalten, muss der Wert auf `None` gesetzt werden;

In [ ]:
from collections import defaultdict

class GameState:
    def __init__(self,
                 state = s0,
                 player = PLAYER_1,
                 algorithm1 = None,
                 algorithm2 = None,
                 timeout = None,
                 stepwise = False,
                 limitMovesWithoutMill = 30,
                 limitStatesCounter = 5):
        
        self.algorithm1 = algorithm1
        self.algorithm2 = algorithm2
        
        self.state = state
        self.player = player
        self.canvas = setupCanvas()
        self.winner = None
        self.information = None
        self.resetStateVariables()

        self.timeout = timeout
        self.stepwise = stepwise
        self.limitMovesWithoutMill = limitMovesWithoutMill
        self.movesWithoutMill = 0
        self.limitStatesCounter = limitStatesCounter
        self.statesCounter = defaultdict(int)
        
        self.pause = (self.player == PLAYER_1 and self.algorithm1) or (self.player == PLAYER_2 and self.algorithm2)
        if self.pause:
            self.hint = 'Please click to start the game.'
        
        self.canvas[2].on_mouse_up(self.handleGame)
        
        updateGui(self.canvas[2], self.state)
        logger.info('game state initalized')
        self.updateText()       

Die Funktion `resetStateVariables` dient zum Zurücksetzen der temporären Hilfsvariablen der Klasse `GameState`. Diese Funktion hat weder Ein- noch Ausgabe.

In [ ]:
def resetStateVariables(self):
    self.stateTemp = None
    self.millsToPound = 0
    self.selectedStone = None
    self.hint = None
    
GameState.resetStateVariables = resetStateVariables
del resetStateVariables

Die Funktion `handleGame` steuert den Ablauf des Spiels. Die Funktion wird bei jedem Mausklick auf das Canvas-Objekt von dem Event `on_mouse_up` aufgerufen. Die Funktion hat zwei Argumente:

- `x` ist relative Wert der Maus zu dem Canvas-Objekt auf der horizontalen Achse;
- `y` ist relative Wert der Maus zu dem Canvas-Objekt auf der vertikalen Achse.

In [ ]:
def handleGame(self, x, y):
    if self.winner is not None:
        logger.warning('Game has ended!')
        return
    
    if self.pause:
        self.pause = False
        self.hint = None
        
        self.updateText()
        self.checkForComputerStep()
    
        return

    phase = playerPhase(self.state, self.player)
    logger.info(f'player phase: {phase}')
    
    stone = getClickedStone(x, y)

    if stone is None:
        logger.warning('No stone was clicked!')
        if self.selectedStone is not None and self.millsToPound <= 0:
            self.cancelStep()
    elif self.millsToPound > 0:
        self.poundMillInGui(stone)
    elif self.selectedStone is not None:
        self.moveStone(stone)
    elif phase == 1:
        self.placeStone(stone)
    elif phase == 2 or phase == 3:
        self.selectStone(stone)

    self.information = None
    self.updateText()

    self.checkForComputerStep()

GameState.handleGame = handleGame
del handleGame

Die Funktion `togglePlayer` tauscht den Spieler, der den nächsten Zug spielt.

In [ ]:
def togglePlayer(self):
    self.player = opponent(self.player)
        
GameState.togglePlayer = togglePlayer
del togglePlayer

Die Funktion `playNewState` spielt einen vollständigen Zug in der GUI. Dafür hat sie ein Argument:

- `newState` ist der neue Zustand, der gespielt werden soll.

Die Funktion aktualisiert alle benötigten Hilfsvariablen in der Klasse `GameState` und ruft diverse Funktionen auf, um das Spiel für den neuen Zug vorzubereiten.

Sind die Regeln für ein Untenschieden `limitStatesCounter` und `limitMovesWithoutMill` aktiviert (also nicht `None`), werden die entsprechenden Zähler, um die Regeln zu kontrollieren, aktualisiert. Die Regeln werden jedoch nur aktualisiert, wenn sich die Spieler nicht mehr in der Setzphase befinden.

In [ ]:
def playNewState(self, newState): 
    movedStone, poundedStones = getChangedStones(self.state, newState, self.player)
    phase = playerPhase(self.state, self.player)
    if phase != 1:
        if self.limitStatesCounter is not None:
            self.statesCounter[newState] += 1
            logger.info(f'the state {newState} was played {self.statesCounter[newState]} times.')

        if self.limitMovesWithoutMill is not None:
            if (len(poundedStones) == 0):
                self.movesWithoutMill += 1
            else:
                self.movesWithoutMill = 0
            logger.info(f'moves without mill: {self.movesWithoutMill}')

    self.state = newState
    logger.info(f'New State was played:\n{newState}')
    self.resetStateVariables()
    self.togglePlayer()
    self.checkIfFinished()
    
    if phase != 1:
        if  (self.limitStatesCounter is not None) and \
            (self.statesCounter[self.state] + 1 >= self.limitStatesCounter):
                self.hint = f'The state has already been played {self.statesCounter[newState]} times. ' \
                            'If it is played once more, the game will end in a remis!'

        if  (self.limitMovesWithoutMill is not None) and \
            (self.movesWithoutMill + 5 >= self.limitMovesWithoutMill):
                self.hint = f'No mill was pound in the last {self.movesWithoutMill} moves. ' \
                            f'In {self.limitMovesWithoutMill - self.movesWithoutMill} moves the game will end in a remis!'
            
    updateGui(self.canvas[2], self.state, movedStone = movedStone, poundedStones = poundedStones)
    self.updateText()

GameState.playNewState = playNewState
del playNewState

Die Funktion `checkForComputerStep` überprüft, ob der nächste Zug von einem Algorithmus gespielt werden soll. Falls dies der Fall ist, wird der jeweilige Algorithmus ausgeführt und der Spielzustand aktualisiert.

In [ ]:
from time import sleep

def checkForComputerStep(self):
    if self.winner is not None:
        logger.warning('Game has ended!')
        return
    
    if self.pause:
        logger.warning('Pause!')
        self.hint = 'The game has paused. Please click to continue!'
        self.updateText()
        return
    
    if (self.player == PLAYER_1 and self.algorithm1) or (self.player == PLAYER_2 and self.algorithm2):
        algorithmName = self.algorithm1.__class__.__name__ \
                        if self.player == PLAYER_1 \
                        else self.algorithm2.__class__.__name__
        logger.info(f'Computer calculating for player {self.player} with algorithm {algorithmName}')
        
        if self.player == PLAYER_1:
            moves = self.algorithm1.bestMoves(self.state, self.player)
        else:
            moves = self.algorithm2.bestMoves(self.state, self.player)
        
        nextState = moves.choice()
        self.information = { 'score': moves.value }
        if moves.debugInformation:
            self.information.update(moves.debugInformation)
            
        logger.info(f'Algorithm {algorithmName} calulated best state with score {moves.value}')
        logger.info(f'Debug Information:\n{moves.debugInformation}')
        self.playNewState(nextState)
        
        if self.timeout:
            self.hint = f'Timeout ({self.timeout} seconds). Please wait!'
            self.updateText()
            sleep(self.timeout)
        
        self.pause = self.stepwise
        self.checkForComputerStep()
    
GameState.checkForComputerStep = checkForComputerStep
del checkForComputerStep

Die Funktion `placeStone` dient zum Platzieren eines Spielersteins in der Spielphase 1. Die Funktion hat ein Argument:

- `coord` ist die Koordinate aus dem Tupel `coords` an dem der Stein des Spielers gesetzt werden soll.

In [ ]:
def placeStone(self, coord):
    if getPlayerAt(self.state[1], coord) != NO_PLAYER:
        logger.warning(f'{coord} is not free')
        self.hint = f'The slot at {coord} is not free!'
        return

    newState = (removeFromStash(self.state[0], self.player), place(self.state[1], coord, self.player))

    if self.validateNewState(newState):
        logging.info('stone placed')
    else:
        logger.info('NewState not in allAvailableStates, checking for new Mills ...')
        self.checkForNewMills(newState)

        
GameState.placeStone = placeStone
del placeStone

Die Funktion `selectStone` dient zum Selektieren des Steines, der in der Phase 2 verschoben bzw. in Phase 3 springen soll. Die Funktion hat ein Argument:

- `stone` ist die Koordinate des Steines, der bewegt werden soll.

Die Funktion erzeugt bei erfolgreicher Validierung einen Hilfszustand in der GUI mit dem markierten Stein.

In [ ]:
def selectStone(self, stone):
    if getPlayerAt(self.state[1], stone) != self.player:
        logger.warning(f'{stone} is not the own stone')
        self.hint = 'Please select your own stone!'
        return
    self.selectedStone = stone
    self.hint = None
    updateGui(self.canvas[2], self.state, selectedStone = self.selectedStone)

    
GameState.selectStone = selectStone
del selectStone

Die Funktion `moveStone` dient zum Bewegen des selektierten Steins in der Zug- und Endphase. Die Funktion hat ein Argument:

- `coord` ist die Koordinate, wohin der Stein bewegt werden soll.

Der zu bewegende Stein wurde in dem vorherigen Hilfszug in der Funktion `selectStone` ausgewählt und in der Hilfsvariablen `selectedStone` gespeichert.

In [ ]:
def moveStone(self, coord):
    if getPlayerAt(self.state[1], coord) != NO_PLAYER:
        logger.warning(f'{coord} is not free')
        self.hint = f'The slot at {coord} is not free!'
        return
    
    canJump = isAllowedToJump(self.state, self.player)

    if canJump or coord in findNeighboringEmptyCells(self.state[1], self.selectedStone):
        newState = (self.state[0], place(self.state[1], self.selectedStone, NO_PLAYER))
        newState = (newState[0], place(newState[1], coord, self.player))

        if self.validateNewState(newState):
            movement = 'jumped' if canJump else 'moved'
            logger.info(f'Stone successfully {movement}!')
        else:
            logger.info('Round not finished, checking for new mills...')
            self.checkForNewMills(newState)
    else:
        logger.warning(f'{coord} is not a (free) neighbor of {self.selectedStone}!')
        self.hint = f'The slot at {coord} is not a (free) neighbor of {self.selectedStone}!'

GameState.moveStone = moveStone
del moveStone

Die Funktion `checkForNewMills` überprüft, ob ein gegebener Zustand neue Mühlen enthält. Die Funktion hat ein Argument:

- `newState` ist der neue Zustand, der überprüft werden soll.

Falls neue Mühlen gefunden worden sind, wird ein temporärer Zustand erstellt, der den menschlichen Spieler auffordert, einen gegnerischen Stein von dem Spiellbrett zu entfernen. Im Englischen wird dies als _pounding_ bezeichnet.

In [ ]:
def checkForNewMills(self, newState):
    oldMills = findMills(self.state[1], self.player)
    newMills = countNewMills(newState[1], oldMills, self.player)

    if newMills > 0:
        self.stateTemp = newState
        self.millsToPound = newMills
        self.hint = None
        movedStone, poundedStones = getChangedStones(self.state, self.stateTemp, self.player)
        updateGui(self.canvas[2], self.stateTemp, movedStone = movedStone, poundedStones = poundedStones)

GameState.checkForNewMills = checkForNewMills
del checkForNewMills

Die Funktion `poundMillInGui` entfernt einen gegnerischen Spielerstein und beendet somit einen Mühlenzug. Die Funktion hat dabei ein Argument:

- `stone` ist die Koordinate des gegnerischen Spielersteins, der entfernt werden soll.

Ob ein Spielerstein entfernt werden kann, wird mit der Funktion `validateNewState` validiert.

In der Setzphase kann es vorkommen, dass ein Spieler zwei Mühlen schlagen kann. In diesem Fall kann das nicht von der Funktion `validateNewState` ausführt werden, weil der Zug noch nicht abgeschlossen ist und somit nicht in der Menge `nextStates` auftritt. In diesem Fall muss die Validierung von der Funktion selber durchgeführt werden.

In [ ]:
def poundMillInGui(self, stone):
    if self.millsToPound  <= 0:
        logger.warning('Player has no Mills to pound!')
        return
    if getPlayerAt(self.state[1], stone) != opponent(self.player):
        logger.warning(f'{stone} is not the opponent!')
        self.hint = 'Please select an opponent stone!'
        
        return
    # if the player has only one mill left to pound, it uses the place function and afterwards
    # validates the newState
    if self.millsToPound == 1:
        newState = (self.stateTemp[0], place(self.stateTemp[1], stone, NO_PLAYER))
        if self.validateNewState(newState):
            logger.info('success')
        else:
            logger.warning('Mills could not be pounded! The new state could not be validated by the game logic.')
            self.hint = 'Please do not select an opponent stone that is in a mill!'

        return
    
    # otherwise the gui has to validate the mill manually,
    # as the intermediate step cannot be checked by the game logic
    if stone in getCellsPoundable(self.stateTemp[1], self.player):
        self.stateTemp = (self.stateTemp[0], place(self.stateTemp[1], stone, NO_PLAYER))
        self.millsToPound -= 1
        self.hint = None
        movedStone, poundedStones = getChangedStones(self.state, self.stateTemp, self.player)
        updateGui(self.canvas[2], self.stateTemp, movedStone = movedStone, poundedStones = poundedStones)
    else:
        logger.warning('Mills could not be pounded! The new state could not be validated by the gui.')
        self.hint = 'Please do not select an opponent stone that is in a mill!'

GameState.poundMillInGui = poundMillInGui
del poundMillInGui

Die Funktion `validateNewState` überprüft, ob ein gegebener Zustand in der Menge der `nextStates` vorhanden ist. Die Funktion hat ein Argument:

- `newState` ist der neue Zustand, der validert werden soll.

Falls sich der neue Zustand `newState` in der Menge `nextStates` von dem aktuellen Zustand `state` befindet, ist ein Zug von dem Spieler abgeschlossen. Die Hilfsvariablen werden zurückgesetzt und es wird der Spieler getauscht. 

In [ ]:
def validateNewState(self, newState):
    allAvailableStates = nextStates(self.state, self.player)
    if newState in allAvailableStates:
        self.playNewState(newState)
        return True
    return False

GameState.validateNewState = validateNewState
del validateNewState

Die Funktion `cancelStep` dient zum Deselektieren eines Steines in der Zug- oder Endphase.

In [ ]:
def cancelStep(self):
    logger.warn('step is canceled.')
    
    self.resetStateVariables()
    updateGui(self.canvas[2], self.state)
    
GameState.cancelStep = cancelStep
del cancelStep

Die Funktion `updateText` dient zum Aktualisieren des Textes und des Hinweises auf dem Spielbrett. Die Funktion ermittelt dabei selbständig den aktuellen Zustand des Spieles anhand der Variablen innerhalb der Klasse.

In [ ]:
def updateText(self):
    phase = playerPhase(self.state, self.player)
    if self.winner is None:
        message = f'Player {self.player}: '
        
        if self.player == PLAYER_1 and self.algorithm1:
            message += f'Computer\'s turn with {self.algorithm1.__class__.__name__}. Please wait.'
        elif self.player == PLAYER_2 and self.algorithm2:
            message += f'Computer\'s turn with {self.algorithm2.__class__.__name__}. Please wait.'
        elif self.millsToPound == 1:
            message += 'Pound your mill.'
        elif self.millsToPound > 1:
            message += f'You have {self.millsToPound} mills left to pound. Please pound your next mill.'
        elif self.selectedStone is not None:
            movement = 'Move' if phase == 2 else 'Jump'
            message += f'{movement} your selected stone.'
        elif phase == 1:
            message += 'Place your stone.'
        elif phase == 2 or phase == 3:
            movement = 'move' if phase == 2 else 'jump'
            message += f'select your stone you want to {movement}.'      
    else:
        message = 'The game has ended: '
        self.hint = None
        if self.winner == NO_PLAYER:
            message += 'Tie.'
        else:
            message += f'{self.winner} has won!'
    logger.info(message)
    drawText(self.canvas[1], message, hint = self.hint, information = self.information)

GameState.updateText = updateText
del updateText

Die Funktion `checkIfFinished` überprüft, ob ein Spiel beendet worden ist. Dabei benutzt es die Funktionen `finished` und `utility` aus dem Jupyter-Notebookt `nmm-game`.

In [ ]:
def checkIfFinished(self):
    if (self.limitMovesWithoutMill is not None) and (self.movesWithoutMill >= self.limitMovesWithoutMill):
        self.winner = NO_PLAYER
    elif (self.limitStatesCounter is not None) and (self.statesCounter[self.state] >= self.limitStatesCounter):
        self.winner = NO_PLAYER
    elif finished(self.state, self.player):
        status = utility(self.state, self.player)
        if status == 0:
            self.winner = NO_PLAYER
        else:
            self.winner = self.player if status == 1 else opponent(self.player)

GameState.checkIfFinished = checkIfFinished
del checkIfFinished

## Spielen

Um das Spiel in der GUI zu spielen, muss zuerst ein Objekt der Klasse `GameState` initialisiert werden. Die Argumente für die Klasse bestimmen die Spieloptionen. Im Folgenden seien vier Beispiele für die Ausführungsoptionen gegeben.

### Beispiel 1

Der weiße Spieler wird von einem Menschen gespielt und der schwarze Spieler von α-β-Pruning mit der Standardkonfiguration.

In [ ]:
gameState = GameState(algorithm2 = AlphaBetaPruningRoteLearning())

gameState.canvas

### Beispiel 2

Der weiße Spieler wird von einem Menschen gespielt und der schwarze Spieler von α-β-Pruning mit maximal 1000 Zuständen pro Zug.

In [ ]:
ab = AlphaBetaPruningRoteLearning()
ab.bestMoves(s0,"w")
# gameState = GameState(algorithm2 = AlphaBetaPruning(max_states=1000))

# gameState.canvas

### Beispiel 3

In diesem Spiel spielt α-β-Pruning für weiß gegen Minimax für schwarz. Beide KI's spielen mit der Standardkonfiguration. Nach jedem Zug gibt es eine automatische Pause von 5 Sekunden.

In [ ]:
# gameState = GameState(algorithm1 = AlphaBetaPruning(), algorithm2 = Minimax(), timeout=5)

# gameState.canvas

### Beispiel 4

In diesem Spiel spielt α-β-Pruning für weiß gegen Minimax für schwarz. α-β-Pruning spielt mit einer geänderten Konfiguration, die auch eine andere Gewichtung der Heuristik verwendet. Das Spiel wird im Einzelschrittmodus gespielt.

In [ ]:
# customWeights = HeuristicWeights(stones = 2, stash = 2, mills = 2, possible_mills = 2)

# gameState = GameState(
#                 algorithm1 = AlphaBetaPruning(max_states = 5_000, weights = customWeights),
#                 algorithm2 = AlphaBetaPruning(),
#                 stepwise = True)

# gameState.canvas